In [1]:
#Классический F-тест и CI для отношения дисперсий

import numpy as np
from scipy.stats import f

np.random.seed(0)
n, m = 20, 18
# сгенерируем две нормальные выборки
X = np.random.normal(loc=0.0, scale=1.5, size=n)
Y = np.random.normal(loc=0.0, scale=1.0, size=m)

Sx2 = X.var(ddof=1)
Sy2 = Y.var(ddof=1)
F_stat = Sx2 / Sy2
df1, df2 = n-1, m-1

# двусторонний p-value: аккуратно
p_one_tail = 1 - f.cdf(F_stat, df1, df2)
p_two_tail = 2 * min(p_one_tail, f.cdf(F_stat, df1, df2))

print("Sx2, Sy2:", Sx2, Sy2)
print("F stat:", F_stat, "p(two-tailed):", p_two_tail)

# доверительный интервал для ratio sigma1^2 / sigma2^2
alpha = 0.05
lower = F_stat / f.ppf(1 - alpha/2, df1, df2)
upper = F_stat / f.ppf(alpha/2, df1, df2)
print("CI for sigma1^2/sigma2^2:", (lower, upper))

Sx2, Sy2: 1.711920301804132 1.6387166603662906
F stat: 1.0446713231203 p(two-tailed): 0.9343013035402521
CI for sigma1^2/sigma2^2: (np.float64(0.39674122336254186), np.float64(2.681664395551634))


Levene и Bartlett (робастность):

In [2]:
from scipy.stats import levene, bartlett

lev_stat, lev_p = levene(X, Y, center='median')  # center='mean' or 'median'
bart_stat, bart_p = bartlett(X, Y)

print("Levene p:", lev_p)
print("Bartlett p:", bart_p)

Levene p: 0.9497361545532691
Bartlett p: 0.9272872991713919


Упражнение 1.1.

In [4]:
np.random.seed(42)
n, m = 20, 18
# сгенерируем две нормальные выборки
X = np.random.normal(loc=0.0, scale=1.5, size=n)
Y = np.random.normal(loc=0.0, scale=1.0, size=m)

Sx2 = X.var(ddof=1)
Sy2 = Y.var(ddof=1)
F_stat = Sx2 / Sy2
df1, df2 = n-1, m-1

# двусторонний p-value: аккуратно
p_one_tail = 1 - f.cdf(F_stat, df1, df2)
p_two_tail = 2 * min(p_one_tail, f.cdf(F_stat, df1, df2))

print("Sx2, Sy2:", Sx2, Sy2)
print("F stat:", F_stat, "p(two-tailed):", p_two_tail)

# доверительный интервал для ratio sigma1^2 / sigma2^2
alpha = 0.05
lower = F_stat / f.ppf(1 - alpha/2, df1, df2)
upper = F_stat / f.ppf(alpha/2, df1, df2)
print("CI for sigma1^2/sigma2^2:", (lower, upper))
print(f'Двусторонний p-value {p_two_tail}')

Sx2, Sy2: 2.073722782434096 0.9672007074007771
F stat: 2.144045973670707 p(two-tailed): 0.11987271199800098
CI for sigma1^2/sigma2^2: (np.float64(0.814257464250977), np.float64(5.5037518717803176))
Двусторонний p-value 0.11987271199800098


Упражнение 2

In [5]:
import numpy as np
from scipy.stats import levene, f

np.random.seed(42)

# Параметры
n_pairs = 2000
n = 20  # размер первой выборки
m = 20  # размер второй выборки

# Параметры логнормального распределения
mu = 0    # среднее в лог-пространстве
sigma = 1  # стандартное отклонение в лог-пространстве

# Счетчики отклонений H0
f_rejections = 0
levene_rejections = 0

for _ in range(n_pairs):
    # Генерируем две выборки из одного логнормального распределения
    sample1 = np.random.lognormal(mu, sigma, n)
    sample2 = np.random.lognormal(mu, sigma, m)
    
    # 1. F-test на равенство дисперсий
    var1 = np.var(sample1, ddof=1)
    var2 = np.var(sample2, ddof=1)
    
    # F-статистика (большая дисперсия / меньшая)
    if var1 >= var2:
        f_stat = var1 / var2
        df1 = n - 1
        df2 = m - 1
    else:
        f_stat = var2 / var1
        df1 = m - 1
        df2 = n - 1
    
    # Двусторонний F-test
    p_f = 2 * min(f.cdf(f_stat, df1, df2), 1 - f.cdf(f_stat, df1, df2))
    if p_f < 0.05:
        f_rejections += 1
    
    # 2. Levene test
    _, p_levene = levene(sample1, sample2, center='median')
    if p_levene < 0.05:
        levene_rejections += 1

# Доли отклонений H0
f_rejection_rate = f_rejections / n_pairs
levene_rejection_rate = levene_rejections / n_pairs

print(f"F-test: доля отклонений H0 = {f_rejection_rate:.4f} (ожидается ~0.05)")
print(f"Levene test: доля отклонений H0 = {levene_rejection_rate:.4f} (ожидается ~0.05)")
print(f"\nF-test отклонений: {f_rejections} из {n_pairs}")
print(f"Levene test отклонений: {levene_rejections} из {n_pairs}")

# Определяем правильный ответ
print("\nЧто верно?")
if abs(f_rejection_rate - 0.05) < 0.01 and abs(levene_rejection_rate - 0.05) < 0.01:
    print("✓ Обе процедуры дают эмпирическое α ≈ 0.05")
elif f_rejection_rate > 0.06 and levene_rejection_rate <= 0.06:
    print("✓ F-test более чувствителен к ненормальности и завышает α")
elif levene_rejection_rate > 0.06 and f_rejection_rate <= 0.06:
    print("✓ Levene test более чувствителен к ненормальности и завышает α")
elif f_rejection_rate > 0.06 and levene_rejection_rate > 0.06:
    print("✓ Обе процедуры сильно завышают α при ненормальности")
else:
    print("Ни одно из утверждений явно не подтверждается")

F-test: доля отклонений H0 = 0.4815 (ожидается ~0.05)
Levene test: доля отклонений H0 = 0.0490 (ожидается ~0.05)

F-test отклонений: 963 из 2000
Levene test отклонений: 98 из 2000

Что верно?
✓ F-test более чувствителен к ненормальности и завышает α


Упражнение 3.1.

In [8]:
import numpy as np
from scipy.stats import levene, bartlett

np.random.seed(42)

n_replications = 1000
sample_size = 30  # размер каждой выборки

# Счетчики отклонений H0 для всех трех сценариев
bartlett_norm = 0
levene_norm = 0

bartlett_lognorm = 0
levene_lognorm = 0

bartlett_second_norm = 0  # третий сценарий тоже нормальные с разными дисперсиями
levene_second_norm = 0

for _ in range(n_replications):
    # 1. Нормальные данные с разными дисперсиями
    sample1_norm = np.random.normal(0, 1, sample_size)  # N(0, 1)
    sample2_norm = np.random.normal(0, 2, sample_size)  # N(0, 4)
    
    # Bartlett test
    _, p_bartlett = bartlett(sample1_norm, sample2_norm)
    if p_bartlett < 0.05:
        bartlett_norm += 1
    
    # Levene test
    _, p_levene = levene(sample1_norm, sample2_norm, center='median')
    if p_levene < 0.05:
        levene_norm += 1
    
    # 2. Логнормальные данные с разными дисперсиями
    # Разные дисперсии достигаем разными сигма в лог-пространстве
    sample1_lognorm = np.random.lognormal(0, 0.5, sample_size)  # меньшая дисперсия
    sample2_lognorm = np.random.lognormal(0, 1.5, sample_size)  # большая дисперсия
    
    _, p_bartlett_lognorm = bartlett(sample1_lognorm, sample2_lognorm)
    if p_bartlett_lognorm < 0.05:
        bartlett_lognorm += 1
    
    _, p_levene_lognorm = levene(sample1_lognorm, sample2_lognorm, center='median')
    if p_levene_lognorm < 0.05:
        levene_lognorm += 1
    
    # 3. Еще раз нормальные данные с разными дисперсиями (другой параметр)
    sample1_norm2 = np.random.normal(5, 1, sample_size)  # N(5, 1) - другое среднее
    sample2_norm2 = np.random.normal(5, 3, sample_size)  # N(5, 9) - большая дисперсия
    
    _, p_bartlett_norm2 = bartlett(sample1_norm2, sample2_norm2)
    if p_bartlett_norm2 < 0.05:
        bartlett_second_norm += 1
    
    _, p_levene_norm2 = levene(sample1_norm2, sample2_norm2, center='median')
    if p_levene_norm2 < 0.05:
        levene_second_norm += 1

# Расчет эмпирических долей
rejection_rate_bartlett_norm = bartlett_norm / n_replications
rejection_rate_bartlett_lognorm = bartlett_lognorm / n_replications
rejection_rate_bartlett_second_norm = bartlett_second_norm / n_replications

# Округление до 1 знака после запятой (требуется для Bartlett в нормальных данных)
rejection_rate_rounded = round(rejection_rate_bartlett_norm, 1)

print("Эмпирические доли отказов H0 (Bartlett):")
print(f"1. Нормальные данные с разными дисперсиями: {rejection_rate_bartlett_norm:.4f} → {rejection_rate_rounded}")
print(f"2. Логнормальные данные с разными дисперсиями: {rejection_rate_bartlett_lognorm:.4f}")
print(f"3. Нормальные данные с разными дисперсиями (второй сценарий): {rejection_rate_bartlett_second_norm:.4f}")

print("\nЭмпирические доли отказов H0 (Levene):")
print(f"1. Нормальные данные с разными дисперсиями: {levene_norm / n_replications:.4f}")
print(f"2. Логнормальные данные с разными дисперсиями: {levene_lognorm / n_replications:.4f}")
print(f"3. Нормальные данные с разными дисперсиями (второй сценарий): {levene_second_norm / n_replications:.4f}")

Эмпирические доли отказов H0 (Bartlett):
1. Нормальные данные с разными дисперсиями: 0.9510 → 1.0
2. Логнормальные данные с разными дисперсиями: 0.9960
3. Нормальные данные с разными дисперсиями (второй сценарий): 1.0000

Эмпирические доли отказов H0 (Levene):
1. Нормальные данные с разными дисперсиями: 0.8920
2. Логнормальные данные с разными дисперсиями: 0.6950
3. Нормальные данные с разными дисперсиями (второй сценарий): 0.9990


Упражнение 4

In [9]:
import numpy as np
from scipy.stats import ttest_ind

np.random.seed(42)

n_simulations = 10000
n = 20  # размер первой выборки
m = 20  # размер второй выборки
sigma1 = 2
sigma2 = 1

# Счетчики ошибок I рода (отклонение H0, когда она верна)
pooled_type1 = 0
welch_type1 = 0

for _ in range(n_simulations):
    # Генерация выборок с одинаковыми средними (μ=0)
    sample1 = np.random.normal(0, sigma1, n)
    sample2 = np.random.normal(0, sigma2, m)
    
    # Pooled t-test (предполагает равные дисперсии)
    t_stat_pooled, p_pooled = ttest_ind(sample1, sample2, equal_var=True)
    if p_pooled < 0.05:
        pooled_type1 += 1
    
    # Welch t-test (не предполагает равенства дисперсий)
    t_stat_welch, p_welch = ttest_ind(sample1, sample2, equal_var=False)
    if p_welch < 0.05:
        welch_type1 += 1

# Частоты ошибок I рода
pooled_type1_rate = pooled_type1 / n_simulations
welch_type1_rate = welch_type1 / n_simulations

print(f"Pooled t-test: {pooled_type1} отклонений из {n_simulations}")
print(f"Частота ошибок I рода: {pooled_type1_rate:.4f} (ожидается ~0.05)")
print(f"\nWelch t-test: {welch_type1} отклонений из {n_simulations}")
print(f"Частота ошибок I рода: {welch_type1_rate:.4f} (ожидается ~0.05)")

# Определяем правильный ответ
print("\nЧто верно о частоте ошибок I/II при многократной симуляции?")

if abs(pooled_type1_rate - 0.05) > 0.01 and abs(welch_type1_rate - 0.05) <= 0.01:
    print("✓ Welch-t контролирует уровень α и более надёжен при неравных σ")
elif abs(pooled_type1_rate - 0.05) <= 0.01 and abs(welch_type1_rate - 0.05) > 0.01:
    print("✓ Pooled-t всегда лучше по мощности")
elif abs(pooled_type1_rate - welch_type1_rate) < 0.005:
    print("✓ Pooled-t и Welch-t дают одинаковые результаты при разных σ")
elif welch_type1_rate > pooled_type1_rate + 0.01:
    print("✓ Welch-t всегда хуже, чем pooled-t")
else:
    print("Результаты требуют дополнительного анализа")

# Для полноты: проверка мощности (ошибки II рода)
# Моделируем с разными средними
power_simulations = 5000
mu_diff = 0.5  # разница в средних

pooled_power = 0
welch_power = 0

for _ in range(power_simulations):
    sample1 = np.random.normal(0, sigma1, n)
    sample2 = np.random.normal(mu_diff, sigma2, m)  # разные средние
    
    _, p_pooled = ttest_ind(sample1, sample2, equal_var=True)
    if p_pooled < 0.05:
        pooled_power += 1
    
    _, p_welch = ttest_ind(sample1, sample2, equal_var=False)
    if p_welch < 0.05:
        welch_power += 1

power_pooled = pooled_power / power_simulations
power_welch = welch_power / power_simulations

print(f"\nМощность тестов (разница средних = {mu_diff}):")
print(f"Pooled t-test мощность: {power_pooled:.4f}")
print(f"Welch t-test мощность: {power_welch:.4f}")

Pooled t-test: 516 отклонений из 10000
Частота ошибок I рода: 0.0516 (ожидается ~0.05)

Welch t-test: 486 отклонений из 10000
Частота ошибок I рода: 0.0486 (ожидается ~0.05)

Что верно о частоте ошибок I/II при многократной симуляции?
✓ Pooled-t и Welch-t дают одинаковые результаты при разных σ

Мощность тестов (разница средних = 0.5):
Pooled t-test мощность: 0.1690
Welch t-test мощность: 0.1630


Упражнение 5

In [ ]:
import numpy as np

np.random.seed(23)

# Создаем две выборки размера n = m = 20
n = 20
m = 20

# Исходные данные (без предположения о распределении)
sample1 = np.random.randn(n) 
sample2 = np.random.randn(m) 
    
# Наблюдаемое отношение дисперсий
var1 = np.var(sample1, ddof=1)
var2 = np.var(sample2, ddof=1)
observed_ratio = max(var1, var2) / min(var1, var2)
observed_stat = abs(observed_ratio - 1)

print(f"Выборка 1: дисперсия = {var1:.3f}")
print(f"Выборка 2: дисперсия = {var2:.3f}")
print(f"Наблюдаемое отношение дисперсий: {observed_ratio:.3f}")
print(f"Наблюдаемая статистика |ratio-1|: {observed_stat:.3f}")

# Объединяем наблюдения
combined = np.concatenate([sample1, sample2])

# Создаем исходные метки: 0 для первой группы, 1 для второй
original_labels = np.array([0] * n + [1] * m)

# Перестановочный тест
B = 5000
count_extreme = 0

for _ in range(B):
    # Случайное распределение меток (перемешивание 0 и 1)
    permuted_labels = np.random.permutation(original_labels)
    
    # Разделяем данные по новым меткам
    group1 = combined[permuted_labels == 0]
    group2 = combined[permuted_labels == 1]
    
    # Вычисляем статистику
    var1_perm = np.var(group1, ddof=1)
    var2_perm = np.var(group2, ddof=1)
    ratio_perm = max(var1_perm, var2_perm) / min(var1_perm, var2_perm)
    stat_perm = abs(ratio_perm - 1)
    
    if stat_perm >= observed_stat:
        count_extreme += 1

# p-value
p_value = count_extreme / B
p_value_rounded = round(p_value, 3)

print(f"\nПерестановочный тест (B={B}):")
print(f"Крайних перестановок: {count_extreme}")
print(f"p-value: {p_value:.5f}")
print(f"p-value (округлено до 3 знаков): {p_value_rounded}")

Выборка 1: дисперсия = 0.942
Выборка 2: дисперсия = 1.040
Наблюдаемое отношение дисперсий: 1.104
Наблюдаемая статистика |ratio-1|: 0.104

Перестановочный тест (B=5000):
Крайних перестановок: 4160
p-value: 0.83200
p-value (округлено до 3 знаков): 0.832
